In [166]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re
import os
import sys
sys.path.insert(0, "/home/katy/CIAlign/CIAlign")
import CIAlign
import utilityFunctions
import importlib
importlib.reload(utilityFunctions)
import copy

In [419]:
def TC_score(benchmark_path, test_path, cleaned=False, cialign_removed=None):
    '''
    Calculates the TC score described in Blackburn and Whealan 2012 to compare a test alignment
    with a benchmark "true" alignment.
    The score is the number of columns which are present in both alignments divided by
    the total number of columns in the true alignment.
    
    A correct column can be at a different position in the alignment but must consist of
    the residues from the same position as in the original alignment
    e.g.
    input
    ATCGG
    AGCC

    benchmark alignment
    ATC-GG
    A-G-CC
    
    test alignment
    A-T-CGG
    AG---CC

    Each non-gap position is given an integer representing its position in the sequence,
    excluding gaps:
    
    benchmark alignment
    ATCGG : 1 2 3 4 5
    A-GCC : 1 0 2 3 4
    
    test alignment
    A-TCGG : 1 0 2 3 4 5
    AG--CC : 1 2 0 0 3 4
    
    Common columns from these sets of integers are then identified
    
    1-1, 4-3 and 5-4 are in both - so 3 columns are present in both alignments.
    
    The benchmark alignment has 5 total columns so the score would be 3/5
    
    Where alignments have been cleaned with CIAlign, ! symbols are inserted to allow this indexing to
    be accurate
    
    e.g.
    input
    ATCGG
    AGCC

    benchmark alignment
    ATC-GG
    A-G-CC
    
    test alignment
    A-T-CGG
    AG---CC
    
    cialign cleaned test alignment
    -T-CGG
    G---CC
    
    stored temporarily as
    !-T-CGG
    !G---CC
    
    '''
    arr_benchmark, nams_benchmark = utilityFunctions.FastaToArray(benchmark_path)
    arr_test, nams_test = utilityFunctions.FastaToArray(test_path)
    if cleaned:
        arr_test = find_removed(cialign_removed, arr_test, nams_test)
    pos_benchmark = np.cumsum(arr_benchmark != "-", 1)
    pos_benchmark[arr_benchmark == "-"] = 0
    pos_test = np.cumsum(arr_test != "-", 1)
    pos_test[arr_test == "-"] = 0
    pos_test[arr_test == "!"] = 9999
    S_orig = set(np.apply_along_axis(lambda x: "_".join(x), 0, pos_benchmark.astype(str)))
    S_new = set(np.apply_along_axis(lambda x: "_".join(x), 0, pos_test.astype(str)))
    return (len(S_new & S_orig), S_new, arr_test, pos_test)

In [420]:
def find_removed(removed_file, arr, nams):
    '''
    Replace nucleotides removed by CIAlign with "!" so that they are counted as mismatches but
    the indices still match the original alignment
    '''
    lines = [line.strip().split("\t") for line in open(removed_file).readlines()]
    D = {x: set() for x in nams}
    for line in lines:
        func = line[0]
        ids = line[-1].split(",")
        if func in ['crop_ends', 'remove_insertions']:
            ids = [int(x) for x in ids]
        if func == "crop_ends":
            nam = line[1]
            D[nam] = D[nam] | set(ids)
        elif func == "remove_insertions":
            for nam in nams:
                if D[nam] != "removed":
                    D[nam] = D[nam] | set(ids)
        elif func in ["remove_divergent", "remove_short"]:
            for nam in ids:
                D[nam] = "removed"

    cleannams = copy.copy(nams)
    cleannams = np.array(cleannams)
    cleanarr = copy.copy(arr)
    for nam, val in D.items():
        which_nam = np.where(cleannams == nam)[0][0]
        if val == "removed":
            cleannams = np.append(cleannams[:which_nam], cleannams[which_nam + 1:])
            cleanarr = np.vstack([cleanarr[:which_nam], cleanarr[which_nam+1:]])
        else:
            which_pos = np.array(sorted(list(val)))
            cleanarr[which_nam, which_pos] = "!"
    cleanarr[arr == "-"] = "-"
    return (cleanarr)

In [421]:
s, s_new, s_test, s_pos = TC_score("human_gapdh_EvolveAGene_nt_true.fasta", "human_gapdh_clustal_auto_EvolveAGene.fasta")

In [422]:
s, s_c, s_test_c, s_pos_c= TC_score("human_gapdh_EvolveAGene_nt_true.fasta", "human_gapdh_clustal_auto_EvolveAGene.fasta", cleaned=True,
          cialign_removed="CIAlign_removed.txt")

In [425]:
s_pos_c[:,181]

array([173, 173, 149, 149, 167, 173, 167, 167, 164, 158, 164, 164, 143,
       143, 146, 146])

In [446]:
one = np.apply_along_axis(lambda x: "_".join(x), 0, s_pos_c.astype(str))
two = np.apply_along_axis(lambda x: "_".join(x), 0, s_pos.astype(str))

In [447]:
np.where(np.array(list(two)) == '0_0_0_0_0_0_0_0_0_0_0_0_0_0_181')

(array([216]),)

In [454]:
s_pos[:, 216]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 181,   0])

In [460]:
s_pos[:, 215:220]

array([[207,   0,   0,   0, 208],
       [207,   0,   0,   0, 208],
       [183,   0,   0,   0, 184],
       [183,   0,   0,   0, 184],
       [201,   0,   0,   0, 202],
       [207,   0,   0,   0, 208],
       [195,   0,   0,   0, 196],
       [195,   0,   0,   0, 196],
       [198,   0,   0,   0, 199],
       [192,   0,   0,   0, 193],
       [198,   0,   0,   0, 199],
       [198,   0,   0,   0, 199],
       [177,   0,   0,   0, 178],
       [177,   0,   0,   0, 178],
       [180, 181, 182, 183, 184],
       [180,   0,   0,   0, 181]])

In [462]:
s_pos_c[:,215:220]

array([[ 207,    0,    0,    0,  208],
       [ 207,    0,    0,    0,  208],
       [ 183,    0,    0,    0,  184],
       [ 183,    0,    0,    0,  184],
       [ 201,    0,    0,    0,  202],
       [ 207,    0,    0,    0,  208],
       [ 195,    0,    0,    0,  196],
       [ 195,    0,    0,    0,  196],
       [ 198,    0,    0,    0,  199],
       [ 192,    0,    0,    0,  193],
       [ 198,    0,    0,    0,  199],
       [ 198,    0,    0,    0,  199],
       [ 177,    0,    0,    0,  178],
       [ 177,    0,    0,    0,  178],
       [ 180, 9999, 9999, 9999,  184],
       [ 180,    0,    0,    0,  181]])

In [430]:
s_new - s_c

{'0_0_0_0_0_0_0_0_0_0_0_0_0_0_181',
 '0_0_0_0_0_0_0_0_0_0_0_0_0_0_182',
 '0_0_0_0_0_0_0_0_0_0_0_0_0_0_183',
 '0_0_0_0_0_0_0_0_0_0_0_0_0_0_743',
 '0_0_0_0_0_0_0_0_0_0_0_0_0_0_744',
 '0_0_0_0_0_0_0_0_0_0_0_0_0_0_745',
 '0_0_0_0_0_0_0_0_0_0_0_773_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_774_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_775_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_776_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_777_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_778_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_779_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_780_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_0_781_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_453_0_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_454_0_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_455_0_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_456_0_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_457_0_0_0_0',
 '0_0_0_0_0_0_0_0_0_0_458_461_0_0',
 '0_0_0_0_0_0_0_0_0_0_459_462_459',
 '0_0_0_0_0_0_0_0_0_0_460_463_460',
 '0_0_0_0_0_0_0_0_0_0_521_524_521',
 '0_0_0_0_0_0_0_0_0_0_522_525_522',
 '0_0_0_0_0_0_0_0_0_0_523_526_523',
 '0_0_0_0_0_0_0_0_0_0_524_527_524',
 '0_0_0_0_0_0_0_0_0_0_525_52

In [341]:
pt[:, 96:100]

array([[  0,   0,   0,  91],
       [  0,   0,   0,  91],
       [  0,   0,   0,  73],
       [  0,   0,   0,  73],
       [ 91,  92,  93,  94],
       [ 97,  98,  99, 100],
       [ 91,  92,  93,  94],
       [ 91,  92,  93,  94],
       [  0,   0,   0,  91],
       [  0,   0,   0,  85],
       [  0,   0,   0,  91],
       [  0,   0,   0,  91],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0]])

In [342]:
tc[:, 96:100]

array([['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['!', '!', '!', 'G'],
       ['!', '!', '!', 'G'],
       ['!', '!', '!', 'G'],
       ['!', '!', '!', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-']], dtype='<U1')

In [335]:
ta[:, 96:100]

array([['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['A', 'T', 'A', 'G'],
       ['T', 'T', 'T', 'G'],
       ['A', 'T', 'C', 'G'],
       ['A', 'T', 'T', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', 'G'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-'],
       ['-', '-', '-', '-']], dtype='<U1')

In [287]:
pc[:,0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [306]:
pt[:,773]

array([  0,   0,   0,   0,   0,   0,   0,   0, 666, 690,   0,   0,   0,
         0,   0,   0])

In [307]:
pc[:,773]

array([749, 749, 715, 724, 740, 752, 713, 728, 719, 743, 706, 718, 724,
       724, 727, 736])

In [237]:
cleanarr = find_removed("CIAlign_removed.txt", arr, nams)

In [304]:
tc[:, 773]

array(['!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!',
       '!', '!', '!'], dtype='<U1')

In [239]:
cleanarr

array([['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['!', '!', '!', ..., 'A', 'A', 'A'],
       ...,
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A']], dtype='<U1')

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no


In [209]:
str2

'ATGGGGAAAATGAAGGTCGGAGTTTACTGTTTCGGGAGTCGAGGCAGCCTTGTCACGAGAGCAGCATTTAATAGAGGAAAAATTTATATCGAGGCAATTAACTTTATAGTATATATGGTTAGAGACGAGTCTACCCATGGTAGATTTCATGGGACCATTGAGGCCGAAATAGGAATTCGTGTTAGCAATGGAAACCCCATTACCATTTATCAGGAACAAGTAAAGTCTAAAATAAATTGTAGCGACGTAGGGGCGGAATATGTAGTAGAATCTACTGGTGTGTTCATAACTATGGAAGAAGCTAGGTCACGTTTATTGGGCGGTGAAAAAAGAGTAATTATATCGGCGCCGTCGGTAGAGGTACCATTGTTTGGTATGGGCGTTAAACAGGAGAAGTACGACAACATTCTAAAATTAATTAGTATACCATCTTTGACAGCGAACCTATTAGCGCTAGTAGCAAAGATTCTCCAGGGAAATGGCTTAATGACTCGAGTTCATGCCATATCAGCGCCACAAAAAACGGTAACGGGTCCAATAGAAAATTTATGGAGGGATGGGAGAGGTGCACTACCCAATATAATCCCTGCATCGACCGGGGCGGCAAAAGCGGGTGGCAAGGTTATCCCAGATCTAAATGGTCAGCGAACCGGTCTGTCTGTTAGTGTGCCAACAGCTAATGTTGCAGTAGTGGCGCTTACCTGCCGCCTCGAAAGACCGGCAAAATATGACGATATCACACAATTAGTGATACAAGCCTCAGAGGGTCCGCTAAAGTGGATCTCAGATTATACTAAATTGCCAGTTGTGTCGTCTGACTTCATTAGTGACAGACATTCTTCAACATTCGATGTCGTCGCAGAACTCGCTTTGAATGTTCACTTCGTAAAATTTATATTTTGGTATGACAATGAAGTAGGATTTAGTAACCGGGTAGGGGATCCAATGGCACACTTGGCGTCTCAAGATTAA'

In [181]:
which_nam which_pos

SyntaxError: invalid syntax (<ipython-input-181-e58342f4565a>, line 1)

In [116]:
arr2[-4, np.array(list(D['A13']))]

IndexError: index 1260 is out of bounds for axis 1 with size 999

In [117]:
np.shape(arr)

(16, 1272)

In [118]:
arr[-4, -100:]

array(['A', 'T', 'T', 'T', 'A', 'T', 'A', 'T', 'C', 'A', 'T', 'G', 'G',
       'T', 'A', 'T', 'G', 'A', 'T', 'A', 'A', 'T', 'G', 'A', 'G', 'G',
       'T', 'T', 'G', 'G', 'G', 'T', 'A', 'T', 'G', 'G', 'G', 'A', 'A',
       'T', 'A', 'G', 'A', 'G', 'T', 'G', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', 'G', 'T', 'C', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'T', 'C', 'A',
       'C', 'A', 'C', 'G', 'T', 'T', 'T', 'A', 'A'], dtype='<U1')

In [122]:
arr[-4, np.array(sorted(list(D['A13'])))]

array(['T', 'C', 'A', 'C', 'A', 'C', 'G', 'T', 'T', 'T', 'A', 'A'],
      dtype='<U1')

In [226]:
arr, nams = utilityFunctions.FastaToArray("human_gapdh_EvolveAGene_nt_true.fasta")

In [227]:
arr2, nams2 = utilityFunctions.FastaToArray("CIAlign_cleaned.fasta")

In [84]:
nams

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16']

In [38]:
nams2

['A1', 'A2', 'A9', 'A14', 'A15', 'A16']

In [39]:
arr

array([['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'A', 'A', 'A'],
       ...,
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A'],
       ['A', 'T', 'G', ..., 'T', 'A', 'A']], dtype='<U1')